In [29]:
# Import necessary libraries
import pandas as pd
import mysql.connector
import numpy as np

In [ ]:
# MySQL database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Test@2025'
}

In [33]:
# Establish connection to MySQL database
conn = mysql.connector.connect(**db_config)

In [34]:
# Function to execute MySQL queries
def execute_query(query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

In [35]:
# Create database if not exists
create_db_query = "CREATE DATABASE IF NOT EXISTS ETL_instances;"
execute_query(create_db_query)

In [36]:
# Use the database
use_db_query = "USE ETL_instances;"
execute_query(use_db_query)

In [37]:
# Create raw_data table
create_raw_data_table_query = """
CREATE TABLE IF NOT EXISTS raw_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT,
    city VARCHAR(255)
);
"""
execute_query(create_raw_data_table_query)

In [38]:
import numpy as np

# Generate sample data with null values and inconsistencies
num_rows = 100
names = ['John', 'Jane', 'Mike', 'Emily', 'Chris', None]  # Including null values
ages = np.random.randint(18, 60, size=num_rows).astype(float)  # Convert to float to allow NaN
nan_indices = np.random.choice(range(num_rows), size=num_rows//10, replace=False)  # Select 10% indices for null values
ages[nan_indices] = np.nan  # Introducing some null values

# Define initial cities list
initial_cities = ['Nairobi', 'Mombasa', 'Kisumu', 'Nakuru', 'Eldoret', '']

# Calculate repetition factor to ensure the list covers num_rows
repetition_factor = num_rows // len(initial_cities) + 1

# Now, extend the list to match num_rows
cities = initial_cities * repetition_factor
np.random.shuffle(cities)  # Shuffle to introduce inconsistencies
cities = cities[:num_rows]  # Trim to match num_rows


In [39]:
# Create DataFrame
sample_df = pd.DataFrame({
    'name': np.random.choice(names, size=num_rows),
    'age': ages,
    'city': cities
})

In [40]:
sample_df.head()

,name,age,city
0,Mike,23.0,Nakuru
1,Jane,NaN,Nakuru
2,Chris,56.0,Mombasa
3,None,25.0,Mombasa
4,Emily,21.0,Eldoret


In [41]:
len(sample_df)

100

In [42]:
# Insert sample data into raw_data table
for index, row in sample_df.iterrows():
    name_value = f"'{row['name']}'" if row['name'] is not None else 'NULL'
    age_value = str(row['age']) if not np.isnan(row['age']) else 'NULL'
    city_value = f"'{row['city']}'" if row['city'] != '' else 'NULL'
    
    insert_query = f"""
    INSERT INTO raw_data (name, age, city) 
    VALUES ({name_value}, {age_value}, {city_value})
    """
    execute_query(insert_query)


In [43]:
# Fetch data from raw_data table and display
fetch_query = "SELECT * FROM raw_data"
fetched_df = pd.read_sql_query(fetch_query, conn)

C:\Users\Amoro\AppData\Local\Temp\ipykernel_6816\1413506267.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fetched_df = pd.read_sql_query(fetch_query, conn)


In [44]:
fetched_df.head()

,id,name,age,city
0,1,Mike,23.0,Nakuru
1,2,Jane,NaN,Nakuru
2,3,Chris,56.0,Mombasa
3,4,None,25.0,Mombasa
4,5,Emily,21.0,Eldoret


In [45]:
#check for missing values
(fetched_df.isna()).sum()

id       0
name    25
age     10
city    17
dtype: int64

In [46]:

fetched_df["city"].unique()

array(['Nakuru', 'Mombasa', 'Eldoret', 'Nairobi', 'Kisumu', None],
      dtype=object)

In [47]:
#summary statistics of continuos variables
fetched_df.describe()

,id,age
count,100.000000,90.000000
mean,50.500000,36.466667
std,29.011492,11.771418
min,1.000000,18.000000
25%,25.750000,26.000000
50%,50.500000,34.500000
75%,75.250000,48.000000
max,100.000000,58.000000


In [48]:
# Perform transformations using Pandas
# Cleaning data - removing rows with missing values
cleaned_df = fetched_df.dropna()

In [49]:
# create a copy of our cleaned df to avoid SettingWithCopyWarning
cleaned_df = cleaned_df.copy()


In [50]:
#Remove rows with empty strings in 'city' column
cleaned_df = cleaned_df[cleaned_df['city'] != '']


In [51]:
#Apply transformations to 'name' column by changing to upper case
cleaned_df.loc[:, 'name'] = cleaned_df.loc[:, 'name'].str.upper()


In [52]:
cleaned_df.head()

,id,name,age,city
0,1,MIKE,23.0,Nakuru
2,3,CHRIS,56.0,Mombasa
4,5,EMILY,21.0,Eldoret
5,6,JOHN,23.0,Nakuru
7,8,CHRIS,50.0,Kisumu


In [55]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      54 non-null     int64 
 1   name    54 non-null     object
 2   age     54 non-null     int64 
 3   city    54 non-null     object
dtypes: int64(2), object(2)
memory usage: 2.1+ KB


In [54]:
# Convert 'age' column to integer type
cleaned_df['age'] = cleaned_df['age'].astype(int)

In [56]:
# Create a new table for cleaned data in MySQL database
create_cleaned_table_query = """
CREATE TABLE IF NOT EXISTS cleaned_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT,
    city VARCHAR(255)
);
"""
execute_query(create_cleaned_table_query)

In [57]:
# Insert cleaned data into MySQL table
for index, row in cleaned_df.iterrows():
    insert_query = f"""
    INSERT INTO cleaned_data (name, age, city) 
    VALUES ('{row['name']}', {row['age']}, '{row['city']}')
    """
    execute_query(insert_query)

In [58]:
# Fetch data from cleaned table and display
fetch_cleaned_query = "SELECT name, age FROM cleaned_data"
cleaned_df_fetched = pd.read_sql_query(fetch_cleaned_query, conn)
print(cleaned_df_fetched.head())

    name  age
0   MIKE   23
1  CHRIS   56
2  EMILY   21
3   JOHN   23
4  CHRIS   50


C:\Users\Amoro\AppData\Local\Temp\ipykernel_6816\1557231860.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cleaned_df_fetched = pd.read_sql_query(fetch_cleaned_query, conn)
